# BDREG - Preparação de Dados

In [4]:
# Gerenciador OS
import os, shutil

import glob

# Módulo gerenciador arquivos compactados
import zipfile

### Filtrar os conteúdos do XML com as categorias de interesse

In [6]:
path_raw = '../../../Data/Raw/'
path_processed = '../../../Data/Processed/'

#Garantir diretório de processamento vazio
for filename in os.listdir(path_processed):
    file_path = os.path.join(path_processed, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Falha ao apagar %s. Razão: %s' % (file_path, e))

print("Extraindo arquivos xml...")
print()

# Iniciando a extração
zip_files = [f for f in glob.glob("".join([path_raw,"*.zip"]), recursive=False)]

for file_file in zip_files:
    with zipfile.ZipFile(file_file, 'r') as zip: 
        # Imprimir conteúdo do ZIP
        zip.printdir() 
        # Extraindo o ZIP 
        print('Extraindo arquivos...') 
        zip.extractall(path = path_processed)
print('Feito!')
print() 

                 2020-09-08 05:50:04         1620
530_20200908_12914804.xml                      2020-09-08 05:46:58         1939
530_20200908_12914812.xml                      2020-09-08 05:47:30         1821
530_20200908_12914813.xml                      2020-09-08 05:44:22         1809
530_20200908_12914814.xml                      2020-09-08 05:41:40         1421
530_20200908_12914815.xml                      2020-09-08 05:41:50         1497
530_20200908_12914816.xml                      2020-09-08 05:40:10         1603
530_20200908_12914817.xml                      2020-09-08 05:39:46         3214
530_20200908_12914818.xml                      2020-09-08 05:39:50         1115
530_20200908_12914819.xml                      2020-09-08 05:39:50         2406
530_20200908_12914820.xml                      2020-09-08 05:41:46         1516
530_20200908_12914821.xml                      2020-09-08 05:43:40         1427
530_20200908_12914822.xml                      2020-09-08 05:45:28    

In [23]:
import bs4
import lxml
import sqlite3
from datetime import datetime 

# Local onde encontra-se o banco de dados da API
path_database = '../db_reg.sqlite3'

categoria_interesse = ['Atos do Congresso Nacional'
                      ,'Atos do Poder Executivo'
                      ,'Presidência da República'
                      ,'Ministério do Desenvolvimento Regional'
                      ,'Ministério de Minas e Energia'
                      ,'Ministério do Meio Ambiente'] 

conteudo = []
insumos_dou = []

# Todos os arquivos extraídos
xml_files = [f for f in glob.glob("".join([path_processed, "*.xml"]), recursive=False)]

# Processa cada arquivo
for file in xml_files:
    print(file)        
    # Read the XML file
    with open(file, "r", encoding='utf-8') as file:
        # Lê cada arquivo, retorna uma lista de linhas
        conteudo = file.readlines()
        
        # Combina linha em string
        conteudo = "".join(conteudo)

        # html.parser sendo utilizado, pois lxml retira informações CDATA
        bs_conteudo = bs4.BeautifulSoup(conteudo, "html.parser")

        #Separar artcategory de interesse
        bs_artcategory = bs_conteudo.find("article").get("artcategory")
           
        e_interesse = [categoria in bs_artcategory for categoria in categoria_interesse]
        
        # Verifica se há categoria de interesse no texto
        if True in e_interesse:
            
            bs_id = bs_conteudo.find("article").get("id")
            bs_nome = bs_conteudo.find("article").get("name")
            bs_secao = bs_conteudo.find("article").get("pubname")
            bs_data_publicacao = bs_conteudo.find("article").get("pubdate")
            bs_num_pagina = bs_conteudo.find("article").get("numberpage")
            bs_url_pdf = bs_conteudo.find("article").get("pdfpage")
            bs_id_materia = bs_conteudo.find("article").get("idmateria")
            bs_num_edicao = bs_conteudo.find("article").get("editionnumber")
            bs_objeto = bs_conteudo.identifica.string if bs_conteudo.identifica.string else ''
            bs_ementa = bs_conteudo.ementa.string if bs_conteudo.ementa.string else ''
            bs_text = bs_conteudo.texto.string if bs_conteudo.texto.string else ''
            bs_codinteresse = 'NA'

            #conversao de tipo para date. Inclusão no banco de dados
            bs_data_publicacao = datetime.strptime(bs_conteudo.find("article").get("pubdate"), '%d/%m/%Y')

            # print(bs_conteudo)
            # print('nome: ', bs_nome)
            # print('secao: ', bs_secao)
            # print('datap: ' + bs_data_publicacao)
            # print('categ: '+ bs_artcategory)
            # print('pag: '+ bs_num_pagina)
            # print('url: ' + bs_url_pdf)
            # print('idmateria: '+ bs_id_materia)
            # print('num_edicao: '+ bs_num_edicao)
            # print('texto: '+bs_text)
            # print('objeto: '+ bs_objeto)
            # print('ementa: '+ bs_ementa)
            
            insumo = (bs_id, bs_nome, bs_secao, bs_data_publicacao, bs_artcategory, bs_num_pagina, bs_url_pdf, bs_id_materia, bs_text, bs_num_edicao, bs_objeto, bs_ementa, bs_codinteresse)
            
            insumos_dou.append(insumo)
            
            #print(insumos_dou)exit

#Insere informações no banco de dados em lote            
try:
    # Atenção: Conexões em caminhos errados não gera erro, cria base nova.
    sqliteConnection = sqlite3.connect(path_database)

    cursor = sqliteConnection.cursor()
    print("Conectado com sucesso ao SQLite")
            
    sqlite_insert_query = """INSERT INTO applications_insumodou (id, nome, secao, dat_publicacao, categoria, num_pagina, url_pdf, id_materia, dsc_texto,                               num_edicao, dsc_objeto, dsc_ementa, cod_interesse ) 
                             VALUES 
                             (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) 
                          """

    cursor.executemany(sqlite_insert_query, insumos_dou)
    sqliteConnection.commit()
    
    print("Total de ", cursor.rowcount, " registros inseridos com sucesso")
    
    sqliteConnection.commit()
    
    cursor.close()
               
except sqlite3.Error as error:
    print("Falha ao inserir registros no database", error)
finally:
    if (sqliteConnection):
        sqliteConnection.close()
    print("Conexão com SQLite fechada")      

/Processed/530_20200908_12912039.xml
../../../Data/Processed/530_20200908_12913051.xml
../../../Data/Processed/515_20200908_12912866.xml
../../../Data/Processed/515_20200908_12912996.xml
../../../Data/Processed/530_20200908_12915226.xml
../../../Data/Processed/530_20200908_12913253.xml
../../../Data/Processed/529_20200908_12912793.xml
../../../Data/Processed/529_20200908_12914530.xml
../../../Data/Processed/529_20200908_12914923.xml
../../../Data/Processed/530_20200908_12914010.xml
../../../Data/Processed/530_20200908_12914528.xml
../../../Data/Processed/530_20200908_12911273.xml
../../../Data/Processed/515_20200908_12915229.xml
../../../Data/Processed/530_20200908_12912656.xml
../../../Data/Processed/530_20200908_12912713.xml
../../../Data/Processed/530_20200908_12914011.xml
../../../Data/Processed/530_20200908_12913809.xml
../../../Data/Processed/530_20200908_12913599.xml
../../../Data/Processed/529_20200908_12915343.xml
../../../Data/Processed/530_20200908_12912284.xml
../../../Data